# Meadow analysis

So now we have 15 sets of points representing mushrooms from our plots and 15 accompanying sets of polygons representing grass tufts. How do we test the hypothesis that they are spatially linked? 

## Table of contents:

[Set up our R environment](#setup)

[Finding distances to nearest grass-tuft neighbor of mushrooms](#findingdistances)

[Simulations](#simulations)

[Preparing real data for processing](#processing)

[Visualize digitizations](#visualize)

[Significance testing of real plots](#significance)

<a id='setup'></a>

### Set up our R environment

packages:

In [1]:
library('png')
library('sp')
library('maptools')
library('rgeos')
library('repr')

Checking rgeos availability: TRUE
rgeos version: 0.3-19, (SVN revision 524)
 GEOS runtime version: 3.4.2-CAPI-1.8.2 r3921 
 Linking to sp version: 1.2-3 
 Polygon checking: TRUE 



In [9]:
setwd("/home/daniel/Documents/Bitty_meadow_analysis/analysis")

load our function for digitizing grasses, "digpol()"

In [16]:
source('digpols.R')

Which, by the way, looks like this:

In [18]:
digpols

function () 
{
    require("sp")
    iter <- 1
    an <- "y"
    grasspols <- NULL
    allpols <- list()
    while (an != "n") {
        grass <- locator()
        grasspols <- Polygon(rbind(cbind(grass$x, grass$y), cbind(grass$x[1], 
            grass$y[1])))
        bb <- Polygons(list(grasspols), ID = paste("grass", iter, 
            sep = ""))
        allpols[[iter]] <- bb
        plot(SpatialPolygons(allpols), lwd = 2, add = TRUE)
        an <- readline(prompt = "Another(y/n)? ")
        if (an == "y") {
            iter <- iter + 1
        }
        else {
            break
        }
    }
    cc <- SpatialPolygons(allpols)
    return(cc)
}

<a id='findingdistances'></a>

## Finding distances to nearest grass-tuft neighbor of mushrooms

One of our plots looks like this, after digitizing: 

<img src='p3_06_sample_pols_mush.png'>

(The red triangles represent a mushroom or a cluster of mushrooms)

Let's simplify things a bit, and just deal with a few imaginary grass tufts and a few imaginary mushrooms:

In [50]:
p <- SpatialPoints(cbind(c(25,5,70),c(25,45,70)))

Use our digpols() function to draw some spatial polygons:

In [23]:
g <- digpols()

What does this look like?

In [59]:
options(repr.plot.width = 5, repr.plot.height = 5) ## using repr to control size of R plotter outputs

In [82]:
plot(g, lwd = 2, xlim = c(0,100), ylim = c(0,100), col = 'green')
plot(p, pch = 17, col = 'red', cex = 2, add = TRUE)
axis(1, pos = 0)
axis(2, pos = 0, at = c(0,100), labels = c('',''), lwd.ticks=0)
axis(2, pos = 0, at=seq(0,100, by = 20), lwd=0, lwd.ticks=1)

<img src='testplot1.png'>

We need to know the distance to the nearest grass clump for each mushroom. We can get the distances using [gDistance()](https://cran.r-project.org/web/packages/rgeos/rgeos.pdf). Make sure to use the 'byid=TRUE' option:

In [61]:
dists <- gDistance(g, p, byid=TRUE); dists 

,grass1,grass2,grass3
1,33.46681,55.24240,14.93716
2,17.79230,63.68932,40.28856
3,30.0747643,0.1559771,58.5043044


We can then find the minimums of each row, and average all of these to get a representative statistic for the distances between mushrooms and grasses, in this plot. 

In [62]:
mindists <- apply(dists, MARGIN = 1, FUN = min)
avgmindists <- mean(mindists)
mindists

1          2          3 
14.9371591 17.7923043  0.1559771

In [63]:

avgmindists

[1] 10.96181

In this plot, on average, the nearest grass clump is 10.96 cm away from any given mushroom. Let's condense this process into a function:

In [39]:
avgNN <- function(grasses, mushrooms) {
        dists <- gDistance(grasses, mushrooms, byid=TRUE)
        mindists <- apply(dists, MARGIN=1, min)
        avgmindists <- mean(mindists)
        return(avgmindists)}

This will take our digitized mushroom and grass objects and tell us the average nearest neighbor distances between them:

In [66]:
 avgNN(grasses=g, mushrooms=p) 

[1] 10.96181

<a id='simulations'></a>

## Simulations

Now we test how often we get this distance or closer with the same spatial distribution of grass clumps, but with the same number of mushrooms, placed randomly over the plot. This is known as a "Monte Carlo" method of generating a probability distribution. To create a new set of spatially random mushroom events:

In [ ]:
yardstick = seq(from = 0, to = 100, by = 0.01)

xx <- sample(yardstick, size = length(p))
yy <- sample(yardstick, size = length(p))
p.random <- SpatialPoints(cbind(xx,yy))

We can condense turn this process into another function, that makes random sets of mushrooms of equal length to our actual mushrooms object:

In [38]:
mushrooms.random <- function(mushrooms){
        yardstick <- seq(from = 0,to = 100, by = 0.01)
        xx <- sample(yardstick, size = length(mushrooms))
        yy <- sample(yardstick, size = length(mushrooms))
        p.random <- SpatialPoints(cbind(xx,yy))
        return(p.random)
}

Try it out:

In [74]:
p.random <- mushrooms.random(mushrooms=p)

What do these look like?

In [84]:
plot(g, lwd = 2, xlim = c(0,100), ylim = c(0,100), col = 'green')
plot(p, pch = 17, col = 'red', cex = 2, add = TRUE)
axis(1, pos = 0)
axis(2, pos = 0, at = c(0,100), labels = c('',''), lwd.ticks=0)
axis(2, pos = 0, at=seq(0,100, by = 20), lwd=0, lwd.ticks=1)
plot(p.random, add=TRUE, col = 'blue', pch = 20, cex = 2)

<img src='testplot2.png'>

(New, random mushrooms are blue dots)

We can use our function from above to see the average distance between nearest mushrooms and grasses in this random set of mushrooms:

In [87]:
avgNN(grasses=g, mushrooms=p.random)

[1] 10.66636

So now we need to generate a lot of these random sets of mushrooms, and see how often they fall as close or closer to grasses. Do this with a function, with an output in the form of  probability seeing an observed average distance result or closer:

In [37]:
getpval <- function(grasses, mushrooms, iterations) {
        avgNN.obs <- avgNN(grasses=grasses, mushrooms=mushrooms)
        avgNN.ran <- NULL
        for(i in 1:iterations){
                avgNN.ran[i] <- avgNN(grasses=grasses, mushrooms = mushrooms.random(mushrooms=mushrooms))}
        pval <- sum(avgNN.ran <= avgNN.obs)/ length(avgNN.ran)
        return(pval)
        }


So for example, with the above set of grasses, mushrooms, and 5000 iterations: 

In [81]:
getpval(grasses=g,mushrooms=p,iterations=5000)

[1] 0.5304

If mushrooms are distributed completely spatially randomly, ~53% of the time we expect to see that mushrooms are as close to grasses as we observed, or closer.

Now we repeat this for all of our actual digitized plots, correct for multiple tests (FDR/Benjamini-Hochberg), and see if a trend emerges.

<a id='processing'></a>

## Preparing real data for processing

Now let's import Megan's digitizations of Bitty's photos. We'lle create a list of these and cycle through them to visualize them, and in the next section, calculate p-values as we did above with a simplified example. What follows is a whole mess of code that is probably not useful to anyone but me, for loading and managing a bunch of photo and R spatial data objects..

In [20]:
digitdir <- '/home/daniel/Documents/Bitty_meadow_analysis/Digitized_Files/' ## shapefiles live here.
photolist <- system('ls /home/daniel/Documents/Bitty_meadow_analysis/photos_mod', intern=TRUE) ## photos that were digitized live here.
shapes <- system(paste('ls',digitdir,sep=' '), intern = TRUE) ## a master list of R spatial data object files


Just to be consistent, the shape files from one our photos was lost, 'T2N' seems to be lacking mushroom and grass shapefiles (sorry Megan!), so let's take this out of the photolist created above:

In [21]:
aa <- which(photolist == 'T2N.png')
photolist <- photolist[-aa]

Let's make a list of just the mushroom point files and just the grass polygon files:

In [22]:
shapes <- system(paste('ls',digitdir,sep=' '), intern = TRUE)
mushfiles <- shapes[grep('mushpts', shapes)]
grassfiles <-shapes[grep('grass', shapes)]

The above code made a list in R out of our directory that contained the mushroom polygons
and grass point files. We can then cycle through them and load all of these files into R. So we'll have all of the grass and mushroom digitized objects in our environment. First mushrooms:

In [10]:
for (i in 1: length(mushfiles)){
load(paste(digitdir,mushfiles[i], sep = ''))
}

In [11]:
ls()

[1] "digitdir"         "grassfiles"       "i"                "mushfiles"       
 [5] "photolist"        "shapes"           "sp.mushpts_p3_01" "sp.mushpts_p3_02"
 [9] "sp.mushpts_p3_03" "sp.mushpts_p3_04" "sp.mushpts_p3_05" "sp.mushpts_p3_06"
[13] "sp.mushpts_p3_07" "sp.mushpts_p3_08" "sp.mushpts_p3_09" "sp.mushpts_p3_10"
[17] "sp.mushpts_T1N"   "sp.mushpts_T1S"   "sp.mushpts_T2S"   "sp.mushpts_T3N"  
[21] "sp.mushpts_T3S"

Put all of these into a list, manually. I know there must be a better way to do this, but the way that R pickles its files/objects makes importing data objects from files difficult for me. So:

In [12]:
mushpts <- c(sp.mushpts_p3_01, sp.mushpts_p3_02, sp.mushpts_p3_03, sp.mushpts_p3_04, sp.mushpts_p3_05, sp.mushpts_p3_06, sp.mushpts_p3_07, sp.mushpts_p3_08, sp.mushpts_p3_09, sp.mushpts_p3_10, sp.mushpts_T1N, sp.mushpts_T1S, sp.mushpts_T2S, sp.mushpts_T3N, sp.mushpts_T3S)

Name the items in the list:

In [13]:
names(mushpts) <- c('p3_01', 'p3_02', 'p3_03', 'p3_04', 'p3_05', 'p3_06', 'p3_07', 'p3_08', 'p3_09', 'p3_10', 'T1N', 'T1S', 'T2S', 'T3N', 'T3S')

This lets us refer to a given set of mushroom points by name, like this:

In [15]:
mushpts$T3S

SpatialPoints:
      coords.x1 coords.x2
 [1,]  13.05524 43.180356
 [2,]  23.62859 24.126676
 [3,]  24.91610  8.577331
 [4,]  59.81307 38.531576
 [5,]  60.71696 47.834347
 [6,]  47.79781 75.009225
 [7,]  74.62096  2.446048
 [8,]  81.34476  5.848770
 [9,]  97.92762 72.551631
[10,]  95.31981 87.045117
Coordinate Reference System (CRS) arguments: NA 

Do the same for the grass polygons:

In [16]:
for (i in 1: length(grassfiles)){
load(paste(digitdir,grassfiles[i], sep = ''))
}


grasspols <- c(
p3_01_grass, p3_02_grass, p3_03_grass, p3_04_grass, p3_05_grass,
p3_06_grass, p3_07_grass, p3_08_grass, p3_09_grass, p3_10_grass,
T1N_grass, T1S_grass, T2S_grass, T3N_grass, T3S_grass
)

names(grasspols) <- c( "p3_01","p3_02","p3_03","p3_04",
"p3_05","p3_06","p3_07","p3_08","p3_09",
"p3_10","T1N","T1S","T2S","T3N","T3S")

They should line up in order:

In [24]:
names(grasspols)
names(mushpts)
photolist

[1] "p3_01" "p3_02" "p3_03" "p3_04" "p3_05" "p3_06" "p3_07" "p3_08" "p3_09"
[10] "p3_10" "T1N"   "T1S"   "T2S"   "T3N"   "T3S"

[1] "p3_01" "p3_02" "p3_03" "p3_04" "p3_05" "p3_06" "p3_07" "p3_08" "p3_09"
[10] "p3_10" "T1N"   "T1S"   "T2S"   "T3N"   "T3S"

[1] "P3_01.png" "P3_02.png" "P3_03.png" "P3_04.png" "P3_05.png" "P3_06.png"
 [7] "P3_07.png" "P3_08.png" "P3_09.png" "P3_10.png" "T1N.png"   "T1S.png"  
[13] "T2S.png"   "T3N.png"   "T3S.png"

A manual inspection of these files shows that all digitized files match their photos. Sweet. Now cycle through these and make graphics. A function for creating graphics, in the environment as we have it right now (won't work without all the above code getting things into place):

<a id='visualize'></a>

## Visualize digitizations

In [25]:
Plotshapes <- function(num){
plot(1, type = 'n', xlim = c(0,100), ylim = c(0,100), xlab = '', ylab = '')
plot(mushpts[[num]], lwd = 3, pch = 2, col = 'green', add=TRUE)
plot(grasspols[[num]], lwd = 3, add=TRUE)
}

Let's take a look at one of these, using this function (and plotting the actual photo for comparison side-by-side):

In [31]:
Plotshapes(14)

<img src='T3N_sample2.png'>

The entirety of these cartoons (left side) are available as .png files in the "cartoons" folder of this repository.

<a id='significance'></a>

## Significance testing of real plots

Now we repeat the above example, of testing the likelihood of finding mushrooms that are on average closer to grass tufts than we observed in our plots, with the null assumption of complete spatial randomness (CSR). We use the functions written [above](#simulations).

In [40]:
pvals.meadow <- NULL

for (i in 1:length(mushpts)){
pvals.meadow[[i]] <- getpval(grasses=grasspols[[i]], mushrooms=mushpts[[i]], iterations=10000)
}

Let's look at all 14 p-values:

In [41]:
pvals.meadow

[1] 0.0210 0.0067 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0002 0.0113
[11] 0.0010 0.0001 0.0022 0.0000 0.0000

This is a pretty good example of multiple testing. Let's do a Benjamini-Hochberg correction of these p-values:

In [42]:
p.adjust(pvals.meadow, method="fdr")

[1] 0.0210000000 0.0077307692 0.0000000000 0.0000000000 0.0000000000
 [6] 0.0000000000 0.0000000000 0.0000000000 0.0003000000 0.0121071429
[11] 0.0013636364 0.0001666667 0.0027500000 0.0000000000 0.0000000000

Wow. We soundly reject the null hypothesis of no spatial relation between the grasses and *Mycena* mushrooms. 